In [128]:
import json
import re
from functools import reduce
from copy import deepcopy

In [2]:
with open('nvdcve-1.0-2017.json') as data_source:
    data = json.load(data_source)

cve_items = data["CVE_Items"]

In [ ]:
impacts_mobile = [
    {'part': 'o', 'product': 'android'},
    {'part': 'o', 'product': 'iphone_os'},
    {'target_sw': 'android'},
]

bins = {
    'mobile': [],
    'generic': [],
    'IoT': []
}

In [192]:
class Stack:
    def __init__(s, cid, uri):        
        path = uri.split(':')
        s.hw = []
        s.os = []
        s.app = []
        s.cid = cid
        data = {
            'vendor': path[3],
            'product': path[4],
            'version': path[5],
            'update': path[6],
            'edition': path[7],
            'language': path[8],
            'sw_edition': path[9],
            'target_sw': path[10],
            'target_hw': path[11]
        }
        part = path[2]
        if part == 'h':
            s.hw.append(data)
        elif part == 'o':
            s.os.append(data)
        elif part == 'a':
            s.app.append(data)


def stacks(nodes, cid):
    """Returns the set of stacks implied by the node specification"""
    result = []
    for n in nodes:
        result.append(stacks_(n, cid))
    return result
        

def stacks_(node, cid):
    if node['operator'] == 'AND':
        return stacks_and(node, cid)
    elif node['operator'] == 'OR':
        return stacks_or(node, cid)
    else:
        raise 'unknown operator %s'%node['operator']
        
def stacks_or(node, cid):
    result = []
    for c in node['cpe']:
        result.append(Stack(cid, c['cpe23Uri']))
    return result


def stacks_and(node, cid):
    kids = stacks(node['children'], cid)
    return merge_multistacks(kids)

def merge_multistacks(multistacks):
    return reduce(merge_stacks, multistacks)

def merge_stacks(xx, yy):
    zz = []
    for x in xx:
        for y in yy:
            zz.append(merge_stack(x, y))
    return zz
            
def merge_stack(x, y):
    z = deepcopy(x)
    if y.hw != []:
        z.hw.append(y.hw)
    if y.os != []:
        z.os.append(y.os)
    if y.app != []:
        z.app.append(y.app)
    return z
    
    
the_stacks = []

for item in cve_items:
    # skip entries that do not have a v3 base metric and are not network attacks
    if "baseMetricV3" not in item["impact"]:
        continue
    vector = item["impact"]["baseMetricV3"]["cvssV3"]["attackVector"]
    if vector != "NETWORK" and vector != "ADJACENT":
        continue
    
    the_stacks += stacks(item['configurations']['nodes'], item['cve']['CVE_data_meta']['ID'])
    
flatstacks = [item for sublist in the_stacks for item in sublist]
len(flatstacks)

59455

In [193]:
os_app = list(filter(lambda x: x.os != [] and x.app != [], flatstacks))
os_app_len = len(os_app)
os_app_list = list(map(lambda x: x.__dict__, os_app))
os_app_len

3311

In [187]:
def stackfilter(f):
    def __sf(stack):
        for x in stack.os:
            print(x)
            for k,v in f['os'].items():
                if x[k] in v:
                    return True
        for x in stack.hw:
            for k,v in f['hw'].items():
                if x[k] in v:
                    return True
        for x in stack.app:
            for k,v in f['app'].items():
                if x[k] in v:
                    return True
        return False
    return __sf

In [188]:
mobile_filter = {
    'os': {
        'product': ['andriod', 'iphone_os', 'linux_kernel', 'windows_rt_8.1', 'samsung_mobile']
    },
    'app': {
        'target_sw': ['andriod', 'iphone_os', 'linux_kernel', 'windows_rt_8.1', 'samsung_mobile']
    },
    'hw': {
        
    }
}    

In [213]:
Type = 2
Product = 4
TargetSw = 10
mobile_indicators = [
    {Type: 'o', Product: 'android'},
    {Type: 'o', Product: 'iphone_os'},
    {Type: 'o', Product: 'linux_kernel'},
    {Type: 'o', Product: 'windows_rt_8.1'},
    {Type: 'o', Product: 'samsung_mobile'},
    {Type: 'a', TargetSw: 'android'},
    {Type: 'a', TargetSw: 'iphone_os'},
    {Type: 'a', TargetSw: 'linux_kernel'},
    {Type: 'a', TargetSw: 'windows_rt_8.1'},
    {Type: 'a', TargetSw: 'samsung_mobile'}
]
mobile_indicators

[{2: 'o', 4: 'android'},
 {2: 'o', 4: 'iphone_os'},
 {2: 'o', 4: 'linux_kernel'},
 {2: 'o', 4: 'windows_rt_8.1'},
 {2: 'o', 4: 'samsung_mobile'},
 {2: 'a', 10: 'android'},
 {2: 'a', 10: 'iphone_os'},
 {2: 'a', 10: 'linux_kernel'},
 {2: 'a', 10: 'windows_rt_8.1'},
 {2: 'a', 10: 'samsung_mobile'}]

In [215]:
mobile = []

for item in cve_items:
    # skip entries that do not have a v3 base metric and are not network attacks
    if "baseMetricV3" not in item["impact"]:
        continue
    vector = item["impact"]["baseMetricV3"]["cvssV3"]["attackVector"]
    if vector != "NETWORK" and vector != "ADJACENT":
        continue
        
    for n in item['configurations']['nodes']:
        if ind_match(mobile_indicators, n):
            mobile.append(item['cve']['CVE_data_meta']['ID'])
            break

len(mobile)

445

In [209]:
def ind_match(ind, node):
    if node['operator'] == 'AND':
        for c in node['children']:
            if ind_match(ind, c):
                return True
    elif node['operator'] == 'OR':
        return cpes_match(ind, node['cpe'])

def cpes_match(ind, cpes):
    for c in cpes:
        if cpe_match(ind, c):
            return True
    return False

def cpe_match(ind, cpe):
    for x in ind:
        match = True
        cs = cpe['cpe23Uri'].split(':')
        for k,v in x.items():
            match = match & (v == cs[k])
        if match:
            return True
    return False

In [181]:
mobile_stacks = list(filter(stackfilter(mobile_filter), flatstacks))
len(mobile_stacks)

{'version': '-', 'sw_edition': '*', 'product': 'windows_7', 'target_hw': '*', 'update': 'sp1', 'language': '*', 'target_sw': '*', 'vendor': 'microsoft', 'edition': '*'}
{'version': '-', 'sw_edition': '*', 'product': 'windows_server_2008', 'target_hw': '*', 'update': 'sp2', 'language': '*', 'target_sw': '*', 'vendor': 'microsoft', 'edition': '*'}
{'version': 'r2', 'sw_edition': '*', 'product': 'windows_server_2008', 'target_hw': '*', 'update': 'sp1', 'language': '*', 'target_sw': '*', 'vendor': 'microsoft', 'edition': '*'}
{'version': '-', 'sw_edition': '*', 'product': 'windows_vista', 'target_hw': '*', 'update': 'sp2', 'language': '*', 'target_sw': '*', 'vendor': 'microsoft', 'edition': '*'}
[{'version': '*', 'sw_edition': '*', 'product': 'windows_10', 'target_hw': '*', 'update': '*', 'language': '*', 'target_sw': '*', 'vendor': 'microsoft', 'edition': '*'}]


TypeError: list indices must be integers or slices, not str

In [194]:
os_app_list[47]

{'app': [{'edition': '*',
   'language': '*',
   'product': 'edge',
   'sw_edition': '*',
   'target_hw': '*',
   'target_sw': '*',
   'update': '*',
   'vendor': 'microsoft',
   'version': '-'}],
 'cid': 'CVE-2017-0137',
 'hw': [],
 'os': [[{'edition': '*',
    'language': '*',
    'product': 'windows_10',
    'sw_edition': '*',
    'target_hw': '*',
    'target_sw': '*',
    'update': '*',
    'vendor': 'microsoft',
    'version': '1511'}]]}

In [115]:
bins = {
    'hardware': [],
    'os': [],
    'application': []
}



def parseNode(n):
    results = []
    

for item in cve_items:
    # skip entries that do not have a v3 base metric and are not network attacks
    if "baseMetricV3" not in item["impact"]:
        continue
    vector = item["impact"]["baseMetricV3"]["cvssV3"]["attackVector"]
    if vector != "NETWORK" and vector != "ADJACENT":
        continue
        
    desc = item["cve"]["description"]["description_data"][0]["value"]
    cfg = item["configurations"]["nodes"]
        
    #TODO you really have just about everything you need here
    for node in item['configurations']['nodes']:
        if node['operator'] == 'OR':
            for cpe in node['cpe']:
                path = cpe['cpe23Uri'].split(':')
                vendor = path[3]
                product = path[4]
                target_sw = path[10]
                target_hw = path[11]
        if node['operator'] == 'AND':
            for subnode in node['children']:
                
        
        
    if 'cpe' in item["configurations"]["nodes"][0]:
        for cpe in item["configurations"]["nodes"][0]["cpe"]:
            path = cpe["cpe23Uri"].split(":")

            data = {
                    'vendor': path[3],
                    'product': path[4],
                    'description': desc
                }

            if path[2] == 'h':
                bins['hardware'].append(data)
            elif path[2] == 'o':
                bins['os'].append(data)
            elif path[2] == 'a':
                bins['application'].append(data)
    

In [116]:
len(bins['hardware']), len(bins['os']), len(bins['application'])

(73, 15451, 26290)

In [166]:
s = set()
for x in sorted(bins['os'], key=lambda x: (x['vendor'], x['product'])):
    s.add('%s %s'%(x['vendor'], x['product']))
    
for x in sorted(s):
    print(x)

apple apple_tv
apple iphone_os
apple mac_os_server
apple mac_os_x
apple safari
apple watchos
backbox backbox_linux
canonical ubuntu_linux
ceragon fiberair_ip-10_firmware
cesanta mongoose_os
cisco asyncos
cisco email_security_appliance_firmware
cisco ios
cisco ios_xe
cisco ios_xr
cisco iox
cisco staros
cisco unified_intelligence_center
cisco web_security_appliance
citrix application_delivery_controller_firmware
citrix netscaler_gateway_firmware
citrix xenmobile_server
contiki-os contiki
debian debian_linux
emc isilon_onefs
fedoraproject fedora
fortinet fortianalyzer_firmware
fortinet fortios
fortinet fortiwlc-sd
freebsd freebsd
google android
ibm i
intel active_management_technology_firmware
juniper junos
juniper junos_space
juniper screenos
linux linux_kernel
microsoft windows_10
microsoft windows_7
microsoft windows_8
microsoft windows_8.1
microsoft windows_rt_8.1
microsoft windows_server_2003
microsoft windows_server_2008
microsoft windows_server_2012
microsoft windows_server_2016
mi

In [113]:
pc_software = [
    ('adobe', '.*'),
    ('13thmonkey', 'udfclient')
]

server_hardware = [
    ('21st_centry_insurance')
]

pc_software_rx = list(map(lambda x: (re.compile(x[0]), re.compile(x[1])), pc_software))

def swEffectsPC(x):
    unmatched = 0
    for y in pc_software_rx:
        if y[0].match(x['vendor']) is not None and y[1].match(x['product']) is not None:
            return True
    #print("WARN: %s:%s did not match"%(x['vendor'], x['product']))
    return False

In [114]:
(
    sum(1 for x in bins['hardware'] if hwEffectsPC(x)) +
    sum(1 for x in bins['os'] if osEffectsPC(x)) + 
    sum(1 for x in bins['application'] if swEffectsPC(x))
)

1496

In [84]:
cats = {
    'mobile': [],
    'pc': [],
    'server': [],
    'network': [],
    'iot': []
}

def hwEffectsMobile(x):
    return False

def osEffectsMobile(x):
    return (
        (x['vendor'] == 'google' and x['product'] == 'android') or
        (x['vendor'] == 'apple' and x['product'] == 'iphone_os') or
        (x['vendor'] == 'linux' and x['product'] == 'linux_kernel') or
        (x['vendor'] == 'microsoft' and x['product'].startswith('windows_rt')) or
        (x['vendor'] == 'samsung' and x['product'] == 'samsung_mobile')
    )

#TODO
def hwEffectsPC(x):
    return False

#TODO finish
def osEffectsPC(x):
    return (
        (x['vendor'] == 'apple' and x['product'] == 'mac_os_x') or
        (x['vendor'] == 'apple' and x['product'] == 'safari') or
        (x['vendor'] == 'linux' and x['product'] == 'linux_kernel') or
        (x['vendor'] == 'redhat' and x['product'] == 'enterprise_linux_desktop') or
        (x['vendor'] == 'redhat' and x['product'] == 'enterprise_linux_workstation') or
        (x['vendor'] == 'backbox' and x['product'] == 'backbox_linux') or
        (x['vendor'] == 'debian' and x['product'] == 'debian_linux') or
        (x['vendor'] == 'fedoraproject' and x['product'] == 'fedora') or
        (x['vendor'] == 'canonical' and x['product'] == 'ubuntu_linux') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_10') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_8.1') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_8') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_7') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_vista') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_xp') or
        (x['vendor'] == 'intel' and x['product'] == 'active_management_technology_firmware')
    )

#TODO
def hwEffectsServer(x):
    return False

def osEffectsServer(x):
    return(
        (x['vendor'] == 'apple' and x['product'] == 'mac_os_server') or
        (x['vendor'] == 'canonical' and x['product'] == 'ubuntu_linux') or
        (x['vendor'] == 'cisco' and x['product'] == 'asyncos') or
        (x['vendor'] == 'cisco' and x['product'] == 'email_security_appliance_firmware') or
        (x['vendor'] == 'cisco' and x['product'] == 'unified_intelligence_center') or
        (x['vendor'] == 'cisco' and x['product'] == 'web_security_appliance') or
        (x['vendor'] == 'citrix' and x['product'] == 'application_delivery_controller_firmware') or
        (x['vendor'] == 'citrix' and x['product'] == 'netscaler_gateway_firmware') or
        (x['vendor'] == 'citrix' and x['product'] == 'xenmobile_server') or
        (x['vendor'] == 'debian' and x['product'] == 'debian_linux') or
        (x['vendor'] == 'emc' and x['product'] == 'isilon_onefs') or
        (x['vendor'] == 'fedoraproject' and x['product'] == 'fedora') or
        (x['vendor'] == 'fortinet' and x['product'] == 'fortianalyzer_firmware') or
        (x['vendor'] == 'fortinet' and x['product'] == 'fortios') or
        (x['vendor'] == 'fortinet' and x['product'] == 'fortiwlc-sd') or
        (x['vendor'] == 'freebsd' and x['product'] == 'freebsd') or
        (x['vendor'] == 'ibm' and x['product'] == 'i') or
        (x['vendor'] == 'juniper' and x['product'] == 'junos_space') or
        (x['vendor'] == 'linux' and x['product'] == 'linux_kernel') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_server_2003') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_server_2008') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_server_2012') or
        (x['vendor'] == 'microsoft' and x['product'] == 'windows_server_2016') or
        (x['vendor'] == 'netapp' and x['product'] == 'data_ontap') or
        (x['vendor'] == 'netbsd' and x['product'] == 'netbsd') or
        (x['vendor'] == 'nixos_project' and x['product'] == 'nixos') or
        (x['vendor'] == 'novell' and x['product'] == 'leap') or
        (x['vendor'] == 'openbsd' and x['product'] == 'openbsd') or
        (x['vendor'] == 'opensuse_project' and x['product'] == 'leap') or
        (x['vendor'] == 'oracle' and x['product'] == 'solaris') or
        (x['vendor'] == 'qnap' and x['product'] == 'qts') or
        (x['vendor'] == 'redhat' and x['product'] == 'enterprise_linux') or
        (x['vendor'] == 'redhat' and x['product'] == 'enterprise_linux_server') or
        (x['vendor'] == 'xen' and x['product'] == 'xen')
    )

def hwEffectsNetwork(x):
    return False

def osEffectsNetwork(x):
    return (
        (x['vendor'] == 'ceragon' and x['product'] == 'fiberair_ip-10_firmware') or
        (x['vendor'] == 'cisco' and x['product'] == 'ios') or # dominated by this line
        (x['vendor'] == 'cisco' and x['product'] == 'ios_xe') or
        (x['vendor'] == 'cisco' and x['product'] == 'ios_xr') or
        (x['vendor'] == 'cisco' and x['product'] == 'iox') or
        (x['vendor'] == 'cisco' and x['product'] == 'staros') or
        (x['vendor'] == 'juniper' and x['product'] == 'junos') or
        (x['vendor'] == 'juniper' and x['product'] == 'screenos') or
        (x['vendor'] == 'mikrotik' and x['product'] == 'routeros') or
        (x['vendor'] == 'mimosa' and x['product'] == 'backhaul_radios') or
        (x['vendor'] == 'mimosa' and x['product'] == 'client_radios') or
        (x['vendor'] == 'onosproject' and x['product'] == 'onos') or
        (x['vendor'] == 'paloaltonetworks' and x['product'] == 'pan-os') or
        (x['vendor'] == 'watchguard' and x['product'] == 'fireware') or
        (x['vendor'] == 'zyxel' and x['product'] == 'wre6505_firmware')
    )

#TODO
def hwEffectsIoT(x):
    return False

def osEffectsIoT(x):
    return (
        (x['vendor'] == 'apple' and x['product'] == 'apple_tv') or
        (x['vendor'] == 'apple' and x['product'] == 'watchos') or
        (x['vendor'] == 'contiki-os' and x['product'] == 'contiki') or
        (x['vendor'] == 'openelec' and x['product'] == 'openelec') or
        (x['vendor'] == 'siemens' and x['product'] == 'ruggedcom_rox_i') or
        (x['vendor'] == 'terra-master' and x['product'] == 'tos') or
        (x['vendor'] == 'cesanta' and x['product'] == 'mongoose_os')
    )

In [87]:
(
    sum(1 for x in bins['hardware'] if hwEffectsIoT(x)) +
    sum(1 for x in bins['os'] if osEffectsIoT(x))
)

35

In [88]:
(
    sum(1 for x in bins['hardware'] if hwEffectsNetwork(x)) +
    sum(1 for x in bins['os'] if osEffectsNetwork(x))
)

12904

In [89]:
(
    sum(1 for x in bins['hardware'] if hwEffectsServer(x)) +
    sum(1 for x in bins['os'] if osEffectsServer(x))
)

1626

In [91]:
(
    sum(1 for x in bins['hardware'] if hwEffectsMobile(x)) +
    sum(1 for x in bins['os'] if osEffectsMobile(x))
)

1333

In [ ]:
cats = {
    'generic': [],
    'pc': [],
    'server': [],
    'mobile': [],
    'iot': []
}

device = {
    'arch': '',
    'components': []
}

firmware = {
    'component': ''
}

os = {
    'arch': []
}

software = {
    'os': [],
    'language': ''
}